In [1]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('data/day_filled_train.csv')

In [34]:
data.columns

Index(['acc_id', 'day', 'class_0', 'class_1', 'class_2', 'class_3', 'class_4',
       'class_5', 'class_6', 'class_7', 'etc_cnt', 'num_opponent',
       'pledge_cnt', 'random_attacker_cnt', 'random_defender_cnt',
       'same_pledge_cnt', 'temp_cnt', 'isMajorClass',
       'avg_play_rate_per_pledge', 'char_id', 'combat_char_cnt',
       'combat_play_time', 'p_etc_cnt', 'non_combat_play_time',
       'play_char_cnt', 'pledge_combat_cnt', 'p_random_attacker_cnt',
       'p_random_defender_cnt', 'p_same_pledge_cnt', 'p_temp_cnt',
       'total_combat_cnt_per_pledge', 'total_item_price', 'time_bin_0',
       'time_bin_1', 'time_bin_2', 'time_bin_3', 'type_0', 'type_1',
       'boss_monster', 'death', 'enchant_count', 'exp_recovery', 'fishing',
       'game_money_change', 'npc_kill', 'party_exp', 'playtime',
       'private_shop', 'quest_exp', 'revive', 'solo_exp', 'total_exp',
       'fishing_prop'],
      dtype='object')

In [4]:
df = data.copy()

## 변수명 정리

<style>
    table {
        display: inline-block
    }
</style>

|의미|축약어|
|:---:|:---:|
|activity|a|
|combat|c|
|pledge|p|
|trade|t|
|payment|pa|

접두어로 쓰이는 축약어는 해당 변수가 있던 필드를 의미  
해당 변수가 어느 필드에 대한 정보인지 파악하기 쉽게 하고자 접두어를 붙여봤습니다.  
접두어 없으면 새로 만든 feature.
  
  
|의미|축약어|
|:---:|:---:|
|count|cnt|
|average|avg|
|sum|sum|
|total|tot|

In [18]:
# columns name 필드접두어 추가한 이름으로 변경

df.rename(columns={'class_0':'c_cnt_class0', # c_cnt_class0 : "combat count class0", 해당 날짜에 class0 으로 전투한 횟수
                  'class_1':'c_cnt_class1',
                  'class_2':'c_cnt_class2',
                  'class_3':'c_cnt_class3',
                  'class_4':'c_cnt_class4',
                  'class_5':'c_cnt_class5',
                  'class_6':'c_cnt_class6',
                  'class_7':'c_cnt_class7',
                  'etc_cnt':'c_etc_cnt', 
                  'num_opponent':'c_num_opponent',
                  'pledge_cnt':'c_pledge_cnt',
                  'random_attacker_cnt':'c_random_attacker_cnt',
                  'random_defender_cnt':'c_random_defender_cnt',
                  'same_pledge_cnt':'c_same_pledge_cnt',
                  'temp_cnt':'c_temp_cnt',
                  'avg_play_rate_per_pledge':'avg_play_rate_per_p',
                  'combat_char_cnt':'p_c_char_cnt', # pledge 필드, combat char count
                  'combat_play_time':'p_c_sum_play_time', # pledge 필드, combat sum play time
                  'non_combat_play_time':'p_non_c_sum_play_time',
                  'play_char_cnt':'p_play_char_cnt',
                  'pledge_combat_cnt':'p_c_cnt',
                  'total_combat_cnt_per_pledge':'tot_c_rank_per_p',
                  'total_item_price':'tot_item_price',
                  'time_bin_0':'t_time_bin_0',
                  'time_bin_1':'t_time_bin_1',
                  'time_bin_2':'t_time_bin_2',
                  'time_bin_3':'t_time_bin_3',
                  'type_0':'t_type_0',
                  'type_1':'t_type_1',
                  'death':'a_death_cnt',
                  'enchant_count':'a_enchant_count',
                  'exp_recovery':'a_exp_recovery_cnt',
                  'fishing':'a_fish',
                  'game_money_change':'a_money_change',
                  'npc_kill':'a_npc_kill',
                  'party_exp':'a_party_exp',
                  'playtime':'a_playtime',
                  'private_shop':'a_private_shop',
                  'quest_exp':'a_quest_exp',
                  'revive':'a_revive_cnt',
                  'rich_monster':'a_boss_monster',
                  'solo_exp':'a_solo_exp',
                  'total_exp':'tot_exp',
                  }, inplace=True)


In [6]:
# c_cnt_class(n): 해당 class(직업)으로 전투한 횟수
# NaN -> 전투 안한 날 -> 0으로

cols = ['c_cnt_class0','c_cnt_class1','c_cnt_class2','c_cnt_class3',
         'c_cnt_class4','c_cnt_class5','c_cnt_class6','c_cnt_class7']

df[cols] = df[cols].fillna(0)

In [7]:
# c_etc_cnt: 기타 전투 횟수
# NaN -> 전투 안한 날 -> 0으로

cols = ['c_etc_cnt']
df[cols] = df[cols].fillna(0)

In [8]:
# c_num_opponent: 전투 상대 캐릭터 수
# NaN -> 전투 안한 날 -> 0으로

cols = ['c_num_opponent']
df[cols] = df[cols].fillna(0)

In [9]:
# c_pledge_cnt: 혈맹간 전투에 참여한 횟수
# NaN -> 전투 안한 날 -> 0으로

cols = ['c_pledge_cnt']
df[cols] = df[cols].fillna(0)

In [10]:
# c_random_attacker_cnt: 본인이 막피 공격한 횟수
# c_random_defender_cnt: 본인이 막피공격 받은 횟수
# NaN -> 공격/방어 안한 날 -> 0으로

cols = ['c_random_attacker_cnt', 'c_random_defender_cnt']
df[cols] = df[cols].fillna(0)

In [11]:
# c_same_pledge_cnt: 같은 혈맹원 내에서 전투한 횟수
# NaN -> 전투 안한 날 -> 0으로

cols = ['c_same_pledge_cnt']
df[cols] = df[cols].fillna(0)

In [12]:
# c_temp_cnt: 단발성 전투 횟수

cols = ['c_temp_cnt']
df[cols] = df[cols].fillna(0)

In [13]:
# isMajorClass: 주류 클래스 플레이 비율
# NaN -> 평균 비율로 채우기

cols = ['isMajorClass']
df[cols] = df.groupby('acc_id').isMajorClass.transform(lambda x: x.fillna(x.mean()))

In [14]:
# avg_play_rate_per_p: 해당 유저가 소속된 혈맹의 평균 유저 접속률
# NaN -> 혈맹에 대한 값 -> 평균으로 채우기

cols = ['avg_play_rate_per_p']
df[cols] = df.groupby('acc_id').avg_play_rate_per_p.transform(lambda x: x.fillna(x.mean()))

# 남은 NaN -> 혈맹 들지 않아 평균이 NaN 이었기 때문 -> 0으로 채우기
df[cols] = df[cols].fillna(0)

In [15]:
# char_id: ?? drop 합니다.
df.drop('char_id', axis=1, inplace=True)

In [16]:
# p_c_char_cnt: 혈맹 전투 참여한 혈맹원 수
# p_c_sum_play_time: 혈맹의 전투 캐릭터들의 플레이 시간의 합
# p_non_c_sum_play_time: 혈맹의 비전투 캐릭터들의 플레이 시간의 합
# p_etc_cnt: 혈맹원의 기타 전투 횟수 합
# p_play_char_cnt: 해당 날짜 게임에 접속한 혈맹원 수
# p_c_cnt: 혈맹 간 전투 횟수
# p_random_attacker_cnt: 혈맹원 중 막피 공격한 횟수 합
# p_random_deffender_cnt: 혈맹원 중 막피 공격당한 횟수 합
# p_same_pledge_cnt: 혈맹원끼리의 전투 횟수 합
# p_temp_cnt: 혈맹원들의 단발성 전투 횟수 합

# 혈맹 데이터 NaN? -> 평균으로 대체 -> 개인에 대한 값이 아닌 가입한 혈맹에 대한 정보이기 때문

cols = ['p_c_char_cnt', 'p_c_sum_play_time', 'p_non_c_sum_play_time', 'p_etc_cnt',
        'p_play_char_cnt', 'p_c_cnt', 'p_random_attacker_cnt', 'p_random_defender_cnt', 
        'p_same_pledge_cnt', 'p_temp_cnt']

for col in cols:
    df[col] = df.groupby('acc_id')[col].transform(lambda x: x.fillna(x.mean()))
    
# 남은 NaN -> 혈맹 들지 않아 평균이 NaN -> 0으로.
df[cols] = df[cols].fillna(0)

In [35]:
# tot_c_rank_per_p: 전체 혈맹들 중 해당 혈맹의 전투컨텐츠 즐기는 순위 (1.0: 1위, 0.0: 최하위)
# 혈맹 데이터 NaN -> 평균으로 대체

cols = ['tot_c_rank_per_p']
df[cols] = df.groupby('acc_id')[cols].transform(lambda x: x.fillna(x.mean()))

# 남은 NaN -> 혈맹 들지 않아 평균 NaN -> 0으로.
df[cols] = df[cols].fillna(0)

In [43]:
# tot_item_price: 해당 날짜의 거래 금액 총합
# time_bin_0,1,2,3: 0~6, 6~12, 12~18, 18~24 시간대 거래 횟수
# type_0,1: 0, 해당 날짜 교환창으로 거래한 횟수. 
#           1, 해당 날짜 개인상점으로 거래한 횟수.
# NaN -> 거래 안한 날 -> 0으로.

cols = ['tot_item_price', 't_time_bin_0', 't_time_bin_1', 't_time_bin_2', 't_time_bin_3', 't_type_0', 't_type_1']
df[cols] = df[cols].fillna(0)

In [48]:
# a_boss_monster: 해당 날짜에 보스몬스터 타격(사냥)한 횟수
# NaN -> 타격(사냥) 안한날

cols = ['a_boss_monster']
df[cols] = df[cols].fillna(0)

In [54]:
# a_death_cnt: 해당 날짜의 캐릭터 사망 횟수 (원래 정규화 되어있음)
# a_revive_cnt: 해당 날짜의 캐릭터 부활 횟수 (원래 정규화 되어있음. a_death_cnt 값과 비슷함)
# NaN -> 안죽거나 부활 안한날 -> 0으로

cols = ['a_death_cnt', 'a_revive_cnt']
df[cols] = df[cols].fillna(0)

In [56]:
# a_enchant_count: 7레벨 이상 인챈트 시도 횟수
# a_exp_recovery_cnt: 경험치 복구 횟수
# a_fish: 일일 낚시 시간
# a_money_change: 일일 아데나 변동량
# a_npc_kill: 죽인 몬스터 수
# a_party_exp: 획득한 파티 사냥 경험치
# a_playtime: 일일 플레이 시간
# a_private_shop: 개인상점 운영 시간
# a_quest_exp: 퀘스트 획득 경험치
# a_solo_exp: 솔로 사냥 획득 경험치
# NaN -> (activity 필드) 모두 해당 날짜에 하지 않음을 의미 -> 0으로.

cols = ['a_enchant_count', 'a_exp_recovery_cnt', 'a_fish', 'a_money_change', 
        'a_npc_kill', 'a_party_exp', 'a_playtime', 'a_private_shop', 'a_quest_exp', 'a_solo_exp']
df[cols] = df[cols].fillna(0)

In [58]:
# tot_exp: solo, party, quest exp 의 총합
# fishing_prop: 낚시시간/플레이시간
# NaN -> 해당 날짜에 하지 않음 -> 0으로

cols = ['tot_exp', 'fishing_prop']
df[cols] = df[cols].fillna(0)

In [59]:
df.isnull().sum()

acc_id                   0
day                      0
c_cnt_class0             0
c_cnt_class1             0
c_cnt_class2             0
c_cnt_class3             0
c_cnt_class4             0
c_cnt_class5             0
c_cnt_class6             0
c_cnt_class7             0
c_etc_cnt                0
c_num_opponent           0
c_pledge_cnt             0
c_random_attacker_cnt    0
c_random_defender_cnt    0
c_same_pledge_cnt        0
c_temp_cnt               0
isMajorClass             0
avg_play_rate_per_p      0
p_c_char_cnt             0
p_c_sum_play_time        0
p_etc_cnt                0
p_non_c_sum_play_time    0
p_play_char_cnt          0
p_c_cnt                  0
p_random_attacker_cnt    0
p_random_defender_cnt    0
p_same_pledge_cnt        0
p_temp_cnt               0
tot_c_rank_per_p         0
tot_item_price           0
time_bin_0               0
time_bin_1               0
time_bin_2               0
time_bin_3               0
type_0                   0
type_1                   0
a

In [61]:
# 파일 저장
df.to_csv('data/28day_no_nan_train.csv')